## Import Libraries

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd

## Read Data 

In [2]:
data = pd.read_csv("../Data/selectivemutism_llm_filtered_en.csv")

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977 entries, 0 to 1976
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Result  1977 non-null   object
 1   Text    1977 non-null   object
dtypes: object(2)
memory usage: 31.0+ KB


In [5]:
data = data[data['Text'].apply(lambda x: isinstance(x, str))]

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977 entries, 0 to 1976
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Result  1977 non-null   object
 1   Text    1977 non-null   object
dtypes: object(2)
memory usage: 31.0+ KB


In [7]:
data.head()

,Result,Text
0,yes,"I can speak 'normally' to 6 people. I feel anxious speaking to them, and times where I can't answer still. I worry that I can't shout etc if there was an emergency in public."
1,yes,scared of raising their hand in class...scared to raise your hand
2,yes,I got denied medically transitioning today because of my selective mutism. I feel empty.
3,yes,"I cannot even talk to them with my full voice, even when alone with them. ... I physically cannot. I want to make the progress of speaking with my full voice but there’s just something stopping me every time."
4,yes,I had selective mutism as a kid for 10 years where I was unable to speak outside of my family. I still have mutism tendencies. I'm very quiet and sometimes don't feel comfortable talking or socializing in regular work school situations.


from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Çeviri fonksiyonunu tanımla
def translate_text(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# DataFrame'deki her satır için çeviri işlemi uygula ve yeni bir kolona ekle
data['Translated_Text'] = data['Text'].apply(translate_text)

# Sonuçları göster
print(data)


In [8]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def batch_translate_text(texts, batch_size=8):
    translated_texts = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translated_texts.extend(translated_batch)
    return translated_texts

texts = data['Text'].tolist()
translated_texts = batch_translate_text(texts, batch_size=32)

data['Translated_Text'] = translated_texts

print(data)

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

C:\Users\halilibrahim.hatun\AppData\Local\miniconda3\envs\psynexa_torch_cpu\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\halilibrahim.hatun\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-tc-big-en-tr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/833k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
data.head()

data.to_csv("agoraphobia_translated_to_tr", index=False)

df = pd.read_csv("agoraphobia_translated_to_tr")